In [ ]:
import os
project_dir = "/Volumes/mnt/Workspace/ai-trading-agents"
os.chdir(project_dir)

In [ ]:
get_ipython().run_line_magic('reload_ext', 'autoreload')  # type: ignore
get_ipython().run_line_magic('autoreload', '2')

In [ ]:
import dotenv
dotenv.load_dotenv()

In [ ]:
from src import db
from src.context import build_context
from src.agents.chief_investment_officer.agent import build_chief_investment_officer_agent

In [ ]:
await db.connect()

In [ ]:
run_id = "c8df2387-cd06-4110-aca0-3ee5d8174302"

In [ ]:
await db.prisma.run.update(where={"id": run_id}, data={"status": "RUNNING"})

In [ ]:
context = await build_context(run_id=run_id)

In [ ]:
agent = await build_chief_investment_officer_agent(context)

In [ ]:
await db.prisma.agentmessage.delete_many(
    where={
        "runId": context.run.id,
    }
)
await db.prisma.recommend.delete_many(
    where={
        "runId": context.run.id,
    }
)

await db.prisma.trade.delete_many(
    where={
        "runId": context.run.id,
    }
)

In [ ]:
agent_messages = await db.prisma.agentmessage.find_many(
    where={
        "runId": context.run.id,
    },
    order={
        "createdAt": "asc",
    }
)

In [ ]:
[msg.messages for msg in agent_messages]

In [ ]:
user_prompt = """
As AI Agentic Chief Investment Officer,Your role is to review your portfolio performance,
identify new opportunities, and recommend appropriate actions to optimize portfolio performance aligned with the user's strategy by orchestrating different investment agents.
Now, please review the user's strategy and portfolio performance, and provide your recommendations working with your teams of investment agents.
"""

In [ ]:
response = await agent.ainvoke(
    {"messages": [{"role": "user", "content": user_prompt}]},
    context=context,
    stream_mode="values",
    print_mode="updates",
)

In [ ]:
await db.prisma.run.update(
    where={
        "id": context.run.id,
    },
    data={
        "status": "SUCCESS",
    }
)

In [ ]:
await db.disconnect()